In [ ]:
import pandas as pd


df = p.read_csv('melbourne_data.csv')
pd.set_option('display.max_column', 100)

Il y a 21 colonnes. `setOption()` pour avoir plus de colonnes. NB : Google Callab en affiche 1 seule.

In [ ]:
df.shape   # (13580, 21)
df.isnull().sum()
"""
Suburb              0
Address             0
Rooms               0
Type                0
Price               0
Method              0
SellerG             0
Date                0
Distance            0
Postcode            0
Bedroom2            0
Bathroom            0
Car                62
Landsize            0
BuildingArea     6450
YearBuilt        5375
CouncilArea      1369
Lattitude           0
Longtitude          0
Regionname          0
Propertycount       0
"""

`info()` pour en savoir plus, notament le nombre de colonnes catégorielles.

In [ ]:
df.info()
"""
class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   Suburb         13580 non-null  object
 1   Address        13580 non-null  object
 2   Rooms          13580 non-null  int64
 3   Type           13580 non-null  object
 4   Price          13580 non-null  float64
 5   Method         13580 non-null  object
 6   SellerG        13580 non-null  object
 7   Date           13580 non-null  object
 8   Distance       13580 non-null  float64
 9   Postcode       13580 non-null  float64
 10  Bedroom2       13580 non-null  float64
 11  Bathroom       13580 non-null  float64
 12  Car            13518 non-null  float64
 13  Landsize       13580 non-null  float64
 14  BuildingArea   7130 non-null   float64
 15  YearBuilt      8205 non-null   float64
 16  CouncilArea    12211 non-null  object
 17  Lattitude      13580 non-null  float64
 18  Longtitude     13580 non-null  float64
 19  Regionname     13580 non-null  object
 20  Propertycount  13580 non-null  float64
"""

3 colonnes ont des valeurs manquantes.

`BuildingArea` et `YearBuilt` manque beaucoup de valeurs et `CouncilArea` + 1000.

`nunique()` permet de connaître le nombre de valeur unique.

In [ ]:
df.nunique()
"""
Suburb             314
Address          13378
Rooms                9
Type                 3
Price             2204
Method               5
SellerG            268
Date                58
Distance           202
Postcode           198
Bedroom2            12
Bathroom             9
Car                 11
Landsize          1448
BuildingArea       602
YearBuilt          144
CouncilArea         33
Lattitude         6503
Longtitude        7063
Regionname           8
Propertycount      31
"""

Utiliser `drop` pour retirer les colonnes à valeurs manquantes.**bold text**

In [ ]:
df = df.drop(columns=['BuildingArea', 'YearBuilt', 'CouncilArea'])

df.isnull().sum()
"""
Suburb            0
Address           0
Rooms             0
Type              0
Price             0
Method            0
SellerG           0
Date              0
Distance          0
Postcode          0
Bedroom2          0
Bathroom          0
Car              62
Landsize          0
Lattitude         0
Longtitude        0
Regionname        0
Propertycount     0
"""

Le remplaceemnt des valeurs manquantes à `car` n'apporte pas une bonne corrélation avec le prix. Elle sera retirer.

df = df.drop(columns=['Car'])

Ne manquant plus de valeurs au cade de données, filtrer les colonnes catégorielles avec des valeurs à faible cardinalité (moins de valeurs uniques ; moins de catégories).

In [ ]:
low_cardinality_col = [col for col in df.columns if df[col].dtype == object and df[col].nunique() < 10]
num_col = [col for col in df.columns if df[col].dtype in [int, float]]

Concaténer les deux colonnes ci-dessus. Supprimer la colonne `Price` car c'est la variable cible (y) (= label) et créer l'ensemble de Features (X).

In [ ]:
X = df[low_cardinality_col + num_col].drop(columns='Price')

X[low_cardinality_col].nunique()
"""
Type          3
Method        5
Regionname    8
dtype: int64
"""

3 colonnnes catégorielles dans l'ensemble de Features X à convertir avant de nourrir le modèle de données de toutes les Features.

L'encoder `one-hot Encoder` est inutilisable car ces 3 colonnes n'ont pas d'ordre particulier. Ce sera `labelEncoder`.

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
X['Type'] = le.fit_transform(df['Type'])
X['Method'] = le.fit_transform(df['Method'])
X['Regionname'] = le.fit_transform(df['Regionname'])

L'ensemble est entièrement numérique. La création du modèle est faisable.

La mise à l'échelle dépend du modèle utilisé.  
Les modèles `DecisionTreeRegressor` et `RandomForestRegroessor` ne la nécessitent pas.

Il faut d'abord splitter les 2 ensembles.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=56)
X_train.shape, X_test.shape   # ((10864, 12), (2716, 12))
y_train.shape, y_test.shape   # ((10864,), (2716,))

Créer le modèle de régression d'arbre de décision, puis vérifier la performance via l'erreur absolue moyenne.

In [ ]:
from sklearn.tree import DecisionTreeRegressor

dtr = DecisionTreeRegressor()
dtr.fit(X_train, y_train)
dtr_prediction = dtr.predict(X_test)


from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(y_test, dtr_prediction)
mae   # 221136.588365243

La MAE est mauvaise.  
Essayer le régresseur Random Forest. Une fonction simplifiera le processus.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

def rf_mae(X_train, X_test, y_train, y_test):
    rfr = RandomForestRegressor()
    rfr.fit(X_train, y_train)
    prediction = rfr.predict(X_test)
    rfr_mae = mean_absolute_error(y_test, prediction)
    return rfr_mae

rf_mae(X_train, X_test, y_train, y_test)
# 170687.84809821867                    # Imparfait, mais meilleur que le modèle de régresseur par arbre de décision.